In [268]:
import pandas as pd
from scipy import stats
import numpy as np
from pandas.api.types import CategoricalDtype
import re
from helpers import get_renamed_blocks

In [269]:
STATES = ['in Betrieb', 'Gesetzlich an Stilllegung gehindert', 'Netzreserve',  'Sicherheitsbereitschaft', 'Sonderfall', 'vorläufig stillgelegt', 'stillgelegt', 'Kohlestromvermarktungsverbot']
ENERGIES = ["Kernenergie", "Braunkohle", "Steinkohle", "Erdgas", "Mineralölprodukte", "Abfall", "Biomasse", ""]


In [270]:
pl0 = pd.read_excel("../data/Kraftwerksliste_2021_1.xlsx", skiprows=9)
prtr = pd.read_excel("../data/2021-12-15_PRTR-Deutschland_Freisetzungen.xlsx")

In [271]:
#list(pl0)

In [272]:
#pl0

In [273]:
cols = [10,11,12,14,17,18,19]
plt = get_renamed_blocks(pl0)
plq = plt.drop(plt.columns[cols],axis=1)

In [274]:
#list(plq)

In [275]:
#plt.groupby("energysource").count()

In [276]:
#list(blocks2)

In [277]:
#pl0

In [278]:
#prtr

In [279]:
#pl0 = pd.read_csv("../Kraftwerksliste_2020.csv", error_bad_lines=False, engine="python", encoding="latin1", sep=";", skiprows=9)
#prtr = pd.read_csv("../data/2019-11-25_PRTR-Deutschland_Freisetzungen.csv", error_bad_lines=False, engine="python", encoding="latin1", sep=";", skiprows=0)
#pl0 = pd.read_csv("Kraftwerksliste_CSV.csv", error_bad_lines=False, engine="python")
bm = pd.read_csv("block_plant_mapper.csv", engine="python")

In [280]:
bm['plantid'] = bm['plantid'].apply(lambda x: str(x).replace('/', '_'))

In [281]:
bm.loc[bm.plantid.str.contains("_")]

,plantid,blockid
573,06-04-11_2001178_8_0,BNA0140
574,06-04-11_2001178_8_0,BNA0144
575,06-04-11_2001178_8_1,BNA0139
576,06-04-11_2001178_8_1,BNA0145
577,06-04-11_2001178_8_1,BNA0146
578,06-04-11_2001178_8_2,BNA0141
579,06-04-11_2001178_8_2,BNA1842
580,06-04-11_2001178_8_2,BNA1820
582,06-04-11_2039669_2_0,BNA0147


In [282]:
#prtr.describe()

In [283]:
pl0 = plq.loc[:]

In [284]:
pl0.loc[pl0["blockid"] == "BNA0645"]

,blockid,company,plantname,plz,place,street,federalstate,blockname,initialop,state,energysource,chp,power
1016,BNA0645,Grosskraftwerk Mannheim AG,GKM,68199,Mannheim,Marguerrestraße 1,Baden-Württemberg,Block 7,30256,gesetzlich an Stilllegung gehindert,Steinkohle,Ja,425.0


In [285]:
#pl0.groupby("energysource").count()

In [286]:
#pl0.groupby("state").count()

In [287]:
pl1 = pl0.loc[pl0["energysource"].isin(ENERGIES)]

In [288]:
pl1.groupby("energysource").count()

,blockid,company,plantname,plz,place,street,federalstate,blockname,initialop,state,chp,power
energysource,,,,,,,,,,,,
Abfall,85,86,81,85,85,66,86,54,83,86,85,86
Biomasse,58,75,47,58,58,38,75,17,57,75,54,75
Braunkohle,77,78,76,77,77,26,78,69,77,78,77,78
Erdgas,317,315,306,317,317,170,318,219,297,318,317,318
Kernenergie,9,9,9,9,9,2,9,9,8,9,9,9
Mineralölprodukte,56,57,56,56,56,30,57,47,50,57,55,57
Steinkohle,120,121,119,120,120,75,121,107,120,121,119,121


In [289]:
#pl0.rename(columns={ pl0.columns[10]: "Energieträger_old", pl0.columns[13]: "Energieträger"}, inplace=True)

In [290]:
#pl0

In [291]:
#bm

In [292]:
#pl0.groupby("Energieträger").count()

In [293]:
#list(plt)

In [294]:
plt = pl1.dropna(subset=["blockid"])

In [28]:
plt = plt.astype({"energysource": 'category'})

In [29]:
# remove duplicates blocks in berlin
idxs = plt.index[plt["blockid"].isin(["BNA0083", "BNA0085b"]) & ~plt["state"].isin(["in Betrieb"])].tolist()
plt2 = plt.loc[~plt.index.isin(idxs)]

In [30]:
#plt["Energieträger"] = plt["Energieträger"].astype('category')
#plt["Energieträger"] = plt["Energieträger"].cat.set_categories(["Kernenergie", "Braunkohle", "Steinkohle", "Erdgas", "Mineralölprodukte"], ordered=True)

In [31]:
#plt["Energieträger"] = plt["Energieträger"].astype('category')

In [32]:
#plt.groupby('Energieträger').count()

In [33]:
pl2 = plt2.loc[:]

In [34]:
#pl2.groupby("Energieträger").count()

In [35]:
# plt.dtypes

In [36]:
#pl2 = plt.rename(columns={ pl1.columns[0]: "blockid", pl1.columns[3]: "PLZ", pl1.columns[4]: "Ort", pl1.columns[8]: "Inbetriebnahme", pl1.columns[9]: "Status", pl1.columns[15]: "KWK", pl1.columns[16]: "Nettoleistung"})
#pl3 = pl2.rename(columns={ pl1.columns[1]: "BlockID" })

In [37]:
def to_year(x):
    if type(x) == type(4):
        return x
    elif type(x) == type(2.5):
        return 0 # handle nans
    return int(x[-4:])

In [38]:
def to_year(x):
    dt = pd.to_datetime(x)
    return dt.year

In [39]:
#blocks.loc[blocks.Energieträger == "Kernenergie"]

In [40]:
#pl2

In [41]:
pl2.loc[pl2.blockid == 'BNA0861a', 'initialop'] = 2012
pl2.loc[pl2.blockid == 'BNA1334', 'initialop'] = 2002
pl2.loc[pl2.blockid == 'BNA1141', 'initialop'] = 1970
pl2.loc[pl2.blockid == 'BNA0418', 'initialop'] = 2013
pl2.loc[pl2.blockid == 'BNA1499', 'initialop'] = 1951
pl2.loc[pl2.blockid == 'BNA1502', 'initialop'] = 2013
pl2.loc[pl2.blockid == 'BNA1500', 'initialop'] = 1990
pl2.loc[pl2.blockid == 'BNA1498', 'initialop'] = 1953
pl2.loc[pl2.blockid == 'BNA1260', 'initialop'] = 2013
pl2.loc[pl2.blockid == 'BNA1056', 'initialop'] = 2006
pl2.loc[pl2.blockid == 'BNA1114', 'initialop'] = 2012



pl2.loc[pl2.blockid == 'BNA0413b', 'initialop'] = 2014 # Westfalen D

pl2.loc[pl2.blockid == 'BNA0355', 'initialop'] = 1981 # Grafenrheinfeld


#pl2['Nettoleistung'] = pl2['Nettoleistung'].str.replace('\n','')
#pl2['Nettoleistung'] = pl2[pd.to_numeric(pl2["Nettoleistung"])]
#pl2['Nettoleistung'] = pl2['Nettoleistung'].str.replace(';','.')

In [42]:
#pl2[] = pl2[pd.to_numeric(pl2["Nettoleistung"])]
pl2['initialop'] = pl2['initialop'].apply(to_year)
pl2['power'] = pl2['power'].apply(pd.to_numeric)

In [43]:
#bm[pl2.duplicated(['BlockID'], keep=False)].sort_values("BlockID", ascending=False)

In [44]:
#pl2[pl2.duplicated(['blockid'], keep=False)].sort_values("blockid", ascending=False)

In [45]:
#cols = [10,11,12,14,17,18,19]
#pl3 = pl2.drop(pl2.columns[cols],axis=1)

In [46]:
#pl3.groupby('Unternehmen').max()

In [47]:
#pl3

In [48]:
pl4 = pl2.copy()

In [49]:
def fix_company(company):
    #print(company)
    company_dict = {"RWE": "RWE AG", "Vattenfall": "Vattenfall GmbH", "Uniper": "Uniper SE", "EnBW": "EnBW AG", "Steag": "Steag GmbH", "Nordzucker": "Nordzucker AG", "Lausitz Energie": "LEAG"}
    for key, value in company_dict.items():
        if key in str(company):
            return value
    return company

In [50]:
z = "Endgültig Stillgelegt 2015"

In [51]:
def extract_still(x):
    match = re.findall("[0-9]{4}",x)
    return match[0] if match else np.nan

In [52]:
def extract_state(state):
    if state == "In Betrieb":
        return "in Betrieb"
    elif "Endgültig Stillgelegt" in state:
        return "stillgelegt"
    elif "Vorläufig Stillgelegt" in state:
        return "vorläufig stillgelegt"
    elif "an Stilllegung gehindert" in state:
        return "Gesetzlich an Stilllegung gehindert"
    else:
        return state

In [53]:
extract_state(z)

'stillgelegt'

In [54]:
def fix_kwk(x):
    return "Nein" if x == "nein" else "Ja" if x == "ja" else x

In [55]:
#pl4 = pl3.ap

In [56]:
pl4['company'] = pl4['company'].apply(lambda x: fix_company(x))
pl4['endop'] = pl4['state'].apply(lambda x: extract_still(x))
pl4['state'] = pl4['state'].apply(lambda x: extract_state(x))
pl4['chp'] = pl4['chp'].apply(lambda x: fix_kwk(x))

In [57]:
pl4.sort_values('endop', ascending=False)

,blockid,company,plantname,plz,place,street,federalstate,blockname,initialop,state,energysource,chp,power,endop
2160,BNA0261b,Erlanger Stadtwerke AG,HKW Erlangen,91052,Erlangen,Äußere Bruckerstr. 33,Bayern,K6 DT2,1982.0,stillgelegt,Steinkohle,Ja,17.4,2020
2159,BNA1183,RheinEnergie AG,HKW Merheim,51109,Köln,NaN,Nordrhein-Westfalen,GuD,1970.0,stillgelegt,Erdgas,Ja,15.8,2020
2158,BNA0705,RWE AG,Niederaußem,50129,Bergheim,NaN,Nordrhein-Westfalen,D,1968.0,stillgelegt,Braunkohle,Ja,297.0,2020
2157,BNA0439,Helmstedter Revier GmbH,Buschhaus,38350,Helmstedt,NaN,Niedersachsen,D,1970.0,stillgelegt,Braunkohle,Nein,352.0,2020
2152,BNA0343,Energieversorgung Gera GmbH,Heizkraftwerk Gera-Nord,7546,Gera,Siemensstraße 45,Thüringen,NaN,1996.0,stillgelegt,Erdgas,Ja,74.0,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2038,BNA0413c,RWE AG,Westfalen,59071,Hamm-Uentrop,NaN,Nordrhein-Westfalen,E,1970.0,Kohlestromvermarktungsverbot,Steinkohle,Nein,763.7,NaN
2039,BNA1673,Vattenfall GmbH,Moorburg A,21079,Hamburg,Moorburger Schanze 2,Hamburg,A,1970.0,Kohlestromvermarktungsverbot,Steinkohle,Ja,800.0,NaN
2040,BNA1558,Vattenfall GmbH,Moorburg B,21079,Hamburg,Moorburger Schanze 2,Hamburg,B,2015.0,Kohlestromvermarktungsverbot,Steinkohle,Ja,800.0,NaN
2041,BNA0216a,Steag GmbH,KW Walsum,47179,Duisburg,Dr. Wilhelm-Roelen-Str.129,Nordrhein-Westfalen,Walsum 9,1988.0,Kohlestromvermarktungsverbot,Steinkohle,Ja,370.0,NaN


In [58]:
newdf = pl4.copy()

In [59]:
#newdf.dtypes

In [60]:
newdf["state"] = newdf["state"].astype('category')
newdf["state"] = newdf["state"].cat.set_categories(STATES, ordered=True)

In [61]:
newdf["chp"] = newdf["chp"].astype('category')
newdf["chp"] = newdf["chp"].cat.set_categories(['Ja', 'Nein'], ordered=True)

In [62]:
atest = newdf.groupby("chp")

In [63]:
#newdf.groupby("energysource").count()

In [64]:
atest.count()

,blockid,company,plantname,plz,place,street,federalstate,blockname,initialop,state,energysource,power,endop
chp,,,,,,,,,,,,,
Ja,523,520,500,523,523,324,523,356,506,523,523,523,59
Nein,191,191,190,191,191,80,191,162,180,191,191,191,53


In [65]:
pl3 = newdf

In [66]:
atest = newdf.groupby(["power"]).count()
atest.sort_values("initialop", inplace=True, ascending=False)

In [67]:
#print(atest)

In [68]:
atest = pl3.groupby(["state"]).count()
# atest

In [69]:
# pl3.rename(columns={ pl3.columns[10]: "Energieträger"}, inplace=True)

In [70]:
bm[bm.duplicated(['blockid'], keep=False)].sort_values("blockid", ascending=False)

,plantid,blockid


In [71]:
#bm2 = bm.drop_duplicates()

In [72]:
pl4 = bm.merge(pl3, how="outer", on="blockid")

In [73]:
#pl4[pl4.duplicated(['BlocID'], keep=False)].sort_values("BlockID", ascending=False)

In [178]:
pl5 = pl4[pd.notnull(pl4['blockid'])]
#pl5 = pl4
pl5a = pl5[pd.notnull(pl5['power'])]

In [295]:
pl6 =  pl5a #[pd.notnull(pl5a['plantid'])]

In [296]:
#pl4[pl4['plantid'].isnull()].sort_values(by=["power"])

In [297]:
pl6['plantid'] = pl6['plantid'].apply(lambda x: str(x).replace('/', '_'))

/tmp/ipykernel_607927/118278390.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pl6['plantid'] = pl6['plantid'].apply(lambda x: str(x).replace('/', '_'))


In [298]:
# pl7 = pl6.replace("/", "_")

In [299]:
#pl7 = pl6.apply(str.replace("/", "_"))

In [300]:
#pl6.loc[pl6.Kraftwerksname == "KW Mittelsbüren"]

In [301]:
pl6['initialop'] = pl6['initialop'].apply(pd.to_numeric)

/tmp/ipykernel_607927/4201543385.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pl6['initialop'] = pl6['initialop'].apply(pd.to_numeric)


In [302]:
blocks = pl6.copy()
stammdaten = pl6.copy()

In [303]:
blocks.sort_values("initialop", inplace=True, ascending=False)
# blocks

In [304]:
plants = blocks.loc[:]
plants = plants.dropna(subset=["plantid", "initialop"]) #TODO: remove initialop from here

In [305]:
plants

,plantid,blockid,company,plantname,plz,place,street,federalstate,blockname,initialop,state,energysource,chp,power,endop
724,nan,BNA1950,Vattenfall GmbH,GuD Marzahn,12681,Berlin,NaN,Berlin,GuD Marzahn,2020.0,in Betrieb,Erdgas,Ja,258.000,NaN
140,06-05-500-0342713,BNA1949,Uniper SE,Datteln,45711,Datteln,NaN,Nordrhein-Westfalen,Datteln 4,2020.0,in Betrieb,Steinkohle,Ja,1052.000,NaN
720,nan,BNA1989,EVH GmbH,Block C,6130,Halle (Saale),NaN,Sachsen-Anhalt,Block C,2020.0,in Betrieb,Erdgas,Ja,57.250,NaN
433,06-09-162-0025-0001,BNA1947,BMW AG,Werk 1.5,80937,München,Knorrstraße 147,Bayern,KWK 1-4,2019.0,in Betrieb,Erdgas,Ja,13.600,NaN
432,06-09-162-0025-0001,BNA1946,BMW AG,Werk 1.1,80809,München,Lerchennauerstraße 76,Bayern,KWK 1-3,2019.0,in Betrieb,Erdgas,Ja,13.500,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518,661-114,BNA0856,TWS Thüringer Wärme Service GmbH,HKW Schwarza,7407,Rudolstadt,NaN,Thüringen,NaN,1936.0,in Betrieb,Erdgas,Ja,26.500,NaN
372,664-02,BNA0196,Mitteldeutsche Braunkohlengesellschaft mbH,Deuben,6682,Teuchern,NaN,Sachsen-Anhalt,NaN,1936.0,in Betrieb,Braunkohle,Ja,67.000,NaN
361,06-05-900-0001413,BNA1331,R.D.M. Arnsberg GmbH,Reno De Medici,59821,Arnsberg,Hellefelderstr. 51,Nordrhein-Westfalen,HD - Kraftwerk,1923.0,in Betrieb,Steinkohle,Ja,18.025,NaN
434,661-34,BNA0755a,Kraftwerk Obernburg GmbH,Obernburg,63784,Obernburg,NaN,Bayern,2,1920.0,in Betrieb,Erdgas,Ja,40.000,NaN


In [306]:
#plants_d = plants.loc[plants.duplicated("KraftwerkID", keep=False)]
# plants_d

In [307]:
#plants_c = plants_d.groupby(["KraftwerkID"]).sum()
#plants_count = plants_d.groupby(["KraftwerkID"]).count()

In [308]:
#plants_f = plants_c.reset_index()

In [309]:
#plants_count = plants_count.loc[:, ["blockid"]]

In [310]:
#plants_count = plants_count.reset_index()
# plants_count 

In [311]:
#plants_f = plants_f.merge(plants_count, on="KraftwerkID", how="inner")
# plants_f

In [312]:
#plants[plants.duplicated(['KraftwerkID'], keep=False)].sort_values("BlockID", ascending=False)

In [313]:
plants["energysource"] = plants["energysource"].astype('category')
plants["energysource"] = plants["energysource"].cat.set_categories(ENERGIES, ordered=True)
plants["state"] = plants["state"].astype('category')
plants["state"] = plants["state"].cat.set_categories(STATES, ordered=True)
plants["chp"] = plants["chp"].astype('category')
plants["chp"] = plants["chp"].cat.set_categories(['Ja', 'Nein'], ordered=True)
plants["federalstate"] = plants["federalstate"].astype('category')
plants["federalstate"] = plants["federalstate"].cat.set_categories(['Baden-Württemberg', 'Bayern', 'Berlin', 'Brandenburg', 'Bremen', 'Hamburg', 'Hessen', 'Mecklenburg-Vorpommern', 'Niedersachsen', 'Nordrhein-Westfalen', 'Rheinland-Pfalz', 'Saarland', 'Sachsen', 'Sachsen-Anhalt', 'Schleswig-Holstein', 'Thüringen'])

In [314]:
pl1 = pl0.loc[pl0["energysource"].isin(["Kernenergie", "Erdgas", "Steinkohle", "Braunkohle", "Steinkohle"])]

In [315]:
p_grp = plants.groupby("plantid")

In [316]:
#plants.groupby("Energip_subgrper").max()

In [317]:
p_subgrp = plants.loc[plants['state'].isin(['in Betrieb','Gesetzlich an Stilllegung gehindert'])].groupby('plantid') # TODO: add 

In [318]:
plants_act = pd.DataFrame()
for plantid, group in p_subgrp:
    entry = {}
    entry["plantid"] = plantid
    entry["activepower"] = group["power"].sum()
    plants_act = plants_act.append(entry, ignore_index=True)

In [319]:
#plants_act.sort_values('activepower', ascending=False)

In [320]:
#ap = plants.loc[plants.state == "in Betrieb" | plants.state == "Gesetzlich an Stilllegung gehindert"]

In [321]:
plants_a = pd.DataFrame()
for plantid, group in p_grp:
    #if not plantid == "06-05-300-9046797":
    #    continue
    entry = {}
    #print(entry)
    entry["plantid"] = plantid
    # entry["BlockID"] = group
    try:
        entry["plantname"] = group["plantname"].value_counts().index[0]
    except IndexError:
        entry["plantname"] = np.NaN
    entry["federalstate"] = group["federalstate"].value_counts().index[0]
    entry["energysource"] = group["energysource"].min()
    try:
        entry["chp"] = group["chp"].min()
    except TypeError:
        #print("aneror")
        #print(plantid)
        #print(group['KWK'].count())
        entry["chp"] = ""
    entry["latestexpanded"] = group["initialop"].max()
    entry["initialop"] = group["initialop"].min()
    entry["totalpower"] = group["power"].sum()
    entry["state"] = group["state"].min()
    entry["blockcount"] = group["blockid"].count()
    try:
        entry["company"] = group["company"].value_counts().index[0]
    except IndexError:
        ;
    #print(entry)
    #break
    plants_a = plants_a.append(entry, ignore_index=True)

In [322]:
#plants_a

In [323]:
#TODO: fix this

In [324]:
#plants_a[pd.isnull(plants_a)]

In [325]:
int_list = ['blockcount', 'initialop', 'latestexpanded']
for column in int_list:
    plants_a[column] = plants_a[column].astype(int)
#plant['blockcount'] = pl2['blockcount'].apply(pd.to_numeric)
#pl2['blockcount'] = pl2['blockcount'].apply(pd.to_numeric)
#pl2['blockcount'] = pl2['blockcount'].apply(pd.to_numeric)
#pl2['blockcount'] = pl2['blockcount'].apply(pd.to_numeric)

In [326]:
plants_a.dtypes

plantid            object
plantname          object
federalstate       object
energysource       object
chp                object
latestexpanded      int64
initialop           int64
totalpower        float64
state              object
blockcount          int64
company            object
dtype: object

In [327]:
plants_a["energysource"] = plants_a["energysource"].astype('category')
plants_a["energysource"] = plants_a["energysource"].cat.set_categories(ENERGIES, ordered=True)
plants_a["state"] = plants_a["state"].astype('category')
plants_a["state"] = plants_a["state"].cat.set_categories(STATES, ordered=True)
plants_a["chp"] = plants_a["chp"].astype('category')
plants_a["chp"] = plants_a["chp"].cat.set_categories(['Ja', 'Nein'], ordered=True)
plants_a["federalstate"] = plants_a["federalstate"].astype('category')
plants_a["federalstate"] = plants_a["federalstate"].cat.set_categories(['Baden-Württemberg', 'Bayern', 'Berlin', 'Brandenburg', 'Bremen', 'Hamburg', 'Hessen', 'Mecklenburg-Vorpommern', 'Niedersachsen', 'Nordrhein-Westfalen', 'Rheinland-Pfalz', 'Saarland', 'Sachsen', 'Sachsen-Anhalt', 'Schleswig-Holstein', 'Thüringen'])

In [328]:
plants_a.dtypes

plantid             object
plantname           object
federalstate      category
energysource      category
chp               category
latestexpanded       int64
initialop            int64
totalpower         float64
state             category
blockcount           int64
company             object
dtype: object

In [329]:
plants_a.loc[plants_a.plantid == "666-999"]

,plantid,plantname,federalstate,energysource,chp,latestexpanded,initialop,totalpower,state,blockcount,company


In [330]:
plants_a.loc[plants_a.plantid == "06-00176010435"]

,plantid,plantname,federalstate,energysource,chp,latestexpanded,initialop,totalpower,state,blockcount,company
33,06-00176010435,Staudinger,Hessen,Steinkohle,Ja,1992,1970,1632.0,in Betrieb,4,Uniper SE


In [331]:
plants_a.loc[plants_a.plantid == "06-05-900-0865327"]

,plantid,plantname,federalstate,energysource,chp,latestexpanded,initialop,totalpower,state,blockcount,company
105,06-05-900-0865327,Gersteinwerk,Nordrhein-Westfalen,Steinkohle,Nein,1984,1973,2011.0,in Betrieb,9,RWE AG


In [332]:
#plants_a = plants.drop_duplicates(subset="KraftwerkID")

In [333]:
column_titles = ['plantid', 'plantname', 'federalstate','energysource', 'chp', 'latestexpanded', 'initialop', 'totalpower', 'state', 'blockcount', 'company']
plants_b = plants_a.reindex(columns=column_titles)

In [334]:
#plants_b

In [335]:
list(blocks)

['plantid',
 'blockid',
 'company',
 'plantname',
 'plz',
 'place',
 'street',
 'federalstate',
 'blockname',
 'initialop',
 'state',
 'energysource',
 'chp',
 'power',
 'endop']

In [336]:
#blocks.drop(['place', 'plz', 'street'], axis=1, inplace=True)

In [337]:
blocks[blocks.duplicated(['blockid'], keep=False)].sort_values("blockid", ascending=False)

,plantid,blockid,company,plantname,plz,place,street,federalstate,blockname,initialop,state,energysource,chp,power,endop


In [338]:
#blocks

In [339]:
plants_b.sort_values("initialop", ascending=False, inplace=True)

In [340]:
#drop_list = ['Blockname', 'KWK', 'Nettoleistung', 'Stilllegung', 'Straße und Hausnummer (Standort Kraftwerk)', 'PLZ', 'Ort', 'Nettoleistung']
#plants_g = plants_f.drop(drop_list, axis=1)

In [341]:
#plants_h = plants_g.rename(columns={"KraftwerkID": "plantid", "Gesamtleistung": "totalpower", "Blockzahl": "blockcount", "BlockID": "blockid", "Bundesland": "federalstate", "Energieträger": "energysource", "Inbetriebnahme": "latestexpanded", "Kraftwerksname": "plantname", "Status": "state", "Unternehmen": "company"})

In [342]:
plants_final = plants_b.loc[:]

In [343]:
plants_final.dtypes

plantid             object
plantname           object
federalstate      category
energysource      category
chp               category
latestexpanded       int64
initialop            int64
totalpower         float64
state             category
blockcount           int64
company             object
dtype: object

In [344]:
# plants_final

In [345]:
# stammdaten

In [346]:
# cols = [0,2,3,9,10,11,12]
stammdaten = pl6.copy()
drop_list = ['plantid', 'blockname', 'energysource', 'initialop', 'chp', 'plantname', 'power', 'state', 'endop', 'company']
stammdaten.drop(drop_list, axis=1, inplace=True)

In [347]:
stmp = pl6.copy()
stammdaten2 = stmp.merge(prtr, how="left", left_on="plantid", right_on="kennnummer")

In [348]:
#stammdaten.to_csv("stammdaten.csv", index=False)

In [349]:
#plants_final.to_csv("plants.csv", index=False)

In [350]:
#blocks.to_csv("blocks_3.csv", index=False)

In [351]:
#stammdaten2

In [352]:
DROP_ST = ['blockid', 'jahr', 'kennnummer','betriebsname','betriebsname_2','plz_x','ort','strasse','hausnr','bundesland','flusseinzugsgebiet','taet_nr','taetigkeit','activity','haupttaetigkeit','branche','sector','nace_id','nace_wirtschaftszweig','nace_sector','stoffgruppe','substances_group','schadstoff','pollutant','umweltkompartiment','releases_to','jahresfracht_freisetzung','versehentliche_freisetzung','schadstoff_schwellenwert','einheit','unit','bestimmungsmethode','determination_method','schutzgrund_fracht','confidential_reason_release','schutzgrund_betrieb','confidential_reason_facility']
DROP_ST2 = ['jahr', 'kennnummer','betriebsname','betriebsname_2','plz','ort','strasse','hausnr','bundesland','flusseinzugsgebiet','taet_nr','taetigkeit','activity','haupttaetigkeit','branche','sector','nace_id','nace_wirtschaftszweig','nace_sector','stoffgruppe','substances_group','schadstoff','pollutant','umweltkompartiment','releases_to','jahresfracht_freisetzung','versehentliche_freisetzung','schadstoff_schwellenwert','einheit','unit','bestimmungsmethode','determination_method','schutzgrund_fracht','confidential_reason_release','schutzgrund_betrieb','confidential_reason_facility']
DROP_ST3 = ['jahr', 'kennnummer', 'bundesland', 'flusseinzugsgebiet', "betriebsname", "betriebsname_2", 'taet_nr','taetigkeit','activity','haupttaetigkeit','branche','sector','nace_id','nace_wirtschaftszweig','nace_sector','stoffgruppe','substances_group','schadstoff','pollutant','umweltkompartiment','releases_to','jahresfracht_freisetzung','versehentliche_freisetzung','schadstoff_schwellenwert','einheit','unit','bestimmungsmethode','determination_method','schutzgrund_fracht','confidential_reason_release','schutzgrund_betrieb','confidential_reason_facility']


In [353]:
#DROP_ST = ['jahr']

In [354]:
plants_d1 = plants_final.merge(prtr, how="left", left_on="plantid", right_on="kennnummer")

In [355]:
#plants_d1.dtypes

In [356]:
pld2 = plants_d1.drop_duplicates(['plantid'])

In [357]:
pld3 = pld2.drop(DROP_ST3, axis=1)
plants_final1 = pld3

In [358]:
plants_final2 = plants_final1.merge(plants_act, on="plantid", how='left')

In [359]:
plants_final3 = plants_final2.dropna(subset=["plantid"])

In [360]:
plants_final3.groupby("energysource").count()

,plantid,plantname,federalstate,chp,latestexpanded,initialop,totalpower,state,blockcount,company,plz,ort,strasse,hausnr,geo_lat_wgs84,geo_long_wgs84,activepower
energysource,,,,,,,,,,,,,,,,,
Kernenergie,8,8,8,8,8,8,8,8,8,8,0,0,0,0,0,0,6
Braunkohle,31,30,31,31,31,31,31,31,31,31,29,29,29,29,29,29,26
Steinkohle,69,68,69,69,69,69,69,69,69,69,66,66,66,66,66,66,50
Erdgas,189,180,189,189,189,189,189,189,189,188,118,118,118,118,118,118,170
Mineralölprodukte,20,20,20,20,20,20,20,20,20,20,7,7,7,7,7,7,18
Abfall,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Biomasse,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [361]:
plants_final2[pd.isnull(plants_final2['plantid'])]

,plantid,plantname,federalstate,energysource,chp,latestexpanded,initialop,totalpower,state,blockcount,company,plz,ort,strasse,hausnr,geo_lat_wgs84,geo_long_wgs84,activepower


In [362]:
#stammdaten2

In [363]:
#plants_final2.sort_values("activepower", ascending=False)

In [364]:
#stammdaten2.dtypes

In [365]:
stammdaten3 = stammdaten2.drop(DROP_ST, axis=1)

In [366]:
stammdaten4 = stammdaten3.drop_duplicates(['plantid'])

In [367]:
#list(stammdaten2)

In [368]:
'''


"CREATE TABLE blocks(plantid TEXT, blockid TEXT NOT NULL PRIMARY KEY, blockdescription TEXT, federalstate TEXT, energysource TEXT, initialop INTEGER, chp TEXT,
blockname TEXT, netpower REAL, state TEXT, endop TEXT, company TEXT, FOREIGN KEY (blockid) REFERENCES addresses(blockid) ON DELETE CASCADE);"
662-01|BNA0164|Vattenfall GmbH|Brunsbüttel|Schleswig-Holstein|GT D|0.0|stillgelegt|Mineralölprodukte|Nein|63.5|201

'''


'\n\n\n"CREATE TABLE blocks(plantid TEXT, blockid TEXT NOT NULL PRIMARY KEY, blockdescription TEXT, federalstate TEXT, energysource TEXT, initialop INTEGER, chp TEXT,\nblockname TEXT, netpower REAL, state TEXT, endop TEXT, company TEXT, FOREIGN KEY (blockid) REFERENCES addresses(blockid) ON DELETE CASCADE);"\n662-01|BNA0164|Vattenfall GmbH|Brunsbüttel|Schleswig-Holstein|GT D|0.0|stillgelegt|Mineralölprodukte|Nein|63.5|201\n\n'

In [369]:
blocks2 = blocks[['plantid', 'blockid', 'blockname', 'federalstate', 'energysource', 'initialop', 'chp', 'plantname', 'power', 'state', 'endop', 'company']]

In [370]:
list(blocks2)

['plantid',
 'blockid',
 'blockname',
 'federalstate',
 'energysource',
 'initialop',
 'chp',
 'plantname',
 'power',
 'state',
 'endop',
 'company']

In [371]:
#blocks2.loc['C', 'x'] = "BNA1949"

In [372]:
stammdaten.to_csv("stammdaten_nh.csv", index=False, header=False)
blocks2.to_csv("blocks_nh.csv", index=False, header=False)
plants_final2.to_csv("plants_nh.csv", index=False, header=False)
stammdaten.to_csv("stammdaten.csv", index=False)
blocks2.to_csv("blocks.csv", index=False)
plants_final2.to_csv("plants.csv", index=False)

In [373]:
stammdaten

,blockid,plz,place,street,federalstate
0,BNA1401a,41517,Grevenbroich,Energiestraße 101,Nordrhein-Westfalen
1,BNA1401b,41517,Grevenbroich,Energiestraße 101,Nordrhein-Westfalen
2,BNA0115,4564,Böhlen,Werkstraße,Sachsen
3,BNA0116,4564,Böhlen,Werkstraße,Sachsen
4,BNA0793,32469,Petershagen,NaN,Nordrhein-Westfalen
...,...,...,...,...,...
724,BNA1950,12681,Berlin,NaN,Berlin
725,BNA1948,55120,Mainz,NaN,Rheinland-Pfalz
726,BNA0639,68169,Mannheim,Otto-Hahn-Straße 1,Baden-Württemberg
727,BNA0638,68169,Mannheim,Otto-Hahn-Straße 1,Baden-Württemberg


In [374]:
blocks.loc[blocks["blockid"] == "BNA0645"]

,plantid,blockid,company,plantname,plz,place,street,federalstate,blockname,initialop,state,energysource,chp,power,endop
59,06-08-2948214,BNA0645,Grosskraftwerk Mannheim AG,GKM,68199,Mannheim,Marguerrestraße 1,Baden-Württemberg,Block 7,1970.0,Gesetzlich an Stilllegung gehindert,Steinkohle,Ja,425.0,NaN


In [375]:
#bm2.to_csv("bpm.csv", index=False)

In [376]:
#blocks

In [377]:
#blocks.groupby('Energieträger').count()

In [378]:
#pl4 = blocks.loc[blocks['Energieträger'] == "Mineralölprodukte"]

In [379]:
#pl4

In [380]:
#pd.set_option('display.max_rows', None)

In [381]:
#pl4.loc[pl4['Energieträger'] == "Mineralölprodukte"].sort_values(["Bundesland", "Unternehmen", "Kraftwerksname"])

In [382]:
#plants_final2

In [383]:
blocks2.sort_values('power', ascending=False)[0:20]

,plantid,blockid,blockname,federalstate,energysource,initialop,chp,plantname,power,state,endop,company
355,666-14,BNA0157,KBR,Schleswig-Holstein,Kernenergie,1970.0,Nein,Brokdorf,1410.0,in Betrieb,NaN,PreussenElektra GmbH
357,666-16,BNA0263,KKI 2,Bayern,Kernenergie,1970.0,Nein,Isar 2,1410.0,in Betrieb,NaN,PreussenElektra GmbH
353,666-12,BNA0802,KKP 2,Baden-Württemberg,Kernenergie,1985.0,Nein,Kernkraftwerk Philippsburg 2,1402.0,stillgelegt,2019,EnBW AG
354,666-13,BNA0251,KWG,Niedersachsen,Kernenergie,1970.0,Nein,Grohnde,1360.0,in Betrieb,NaN,PreussenElektra GmbH
358,666-17,BNA0607,KKE,Niedersachsen,Kernenergie,1970.0,Nein,Kernkraftwerk Emsland,1336.0,in Betrieb,NaN,RWE AG
359,666-18,BNA0686,GKN II,Baden-Württemberg,Kernenergie,1989.0,Nein,Gemeinschaftskernkraftwerk Neckarwestheim II,1310.0,in Betrieb,NaN,EnBW AG
352,666-11,BNA0382,C,Bayern,Kernenergie,1970.0,Nein,Kernkraft Gundremmingen,1288.0,in Betrieb,NaN,RWE AG
351,666-11,BNA0381,B,Bayern,Kernenergie,1970.0,Nein,Kernkraft Gundremmingen,1284.0,stillgelegt,2017,RWE AG
356,666-15,BNA0355,KKG,Bayern,Kernenergie,1970.0,Nein,Grafenrheinfeld,1275.0,stillgelegt,2015,E.ON Kernkraft GmbH
0,06-05-100-0248923,BNA1401a,Neurath F,Nordrhein-Westfalen,Braunkohle,1970.0,Nein,BoA 2,1060.0,in Betrieb,NaN,RWE AG
